In [ ]:
#imports

# Events and Event Generation

In the previous notebook, we wrote some of the functions necessary to describe FourMomenta, and to calculate the cross-sections for events.

We're going to build on this here in making a more complex derived data type - `Event`, representing a particular $e^+e^-\rightarrow\mu^+\mu^-$ - and then getting into the meat of our algorithm, an actual event generator.

## Events

The functionality for our Event type is localised to the `events.jl` file. 
As well as describing the data type, we also provide utility functions for instantiating instances from either a Dictionary of FourMomenta (key'd by the particle type), or from a bare set of 4-coordinates.


### The Event type

At the end of the last notebook, we wrote a function to generate a tuple of 4 FourMomenta that characterise a particular $e^+e^-\rightarrow\mu^+\mu^-$ event in terms of the particles involved.

This is almost enough to characterise an event from our Event generator's perspective, but we would also like to add a field representing the weight of this event, according to the generator. 
We'd also like to take advantage of the fact that Julia allows you to override the default constructor for a type to do some validation of our data - that way, we can ensure that if an Event *is* successfully created, it is also a *valid* event.

Appropriate validation checks include:
- the weight of the Event must be >= 0 (probability is positive-definite!)
- the 3-momenta of the electron and positron must be equal and opposite
- the 3-momenta of the muon and anti-muon must be equal and opposite
- the 4-momenta of the electron and muon must have the same energy component (we will assume that a valid Four-momentum is generated for the electron and muon at a given energy)

In [ ]:
struct Event{T}
    electron_momentum::FourMomentum{T}
    #more Fields here 

    weight::T

    #this is an override for the constructor
    function Event(#params here ) where {T<:Real}

        #validation here

        #this just directly makes a new Event via new
        return new{T}(electron_momentum,positron_momentum,muon_momentum,anti_muon_momentum,weight)
    end
end

We should also add, as before, accessor functions for the Event and other methods for constructing them from different parameters.

Obvious functionality includes:
- providing a function to build an Event from E,$\cos\theta$, $\phi$ + weight. This is easy, since we already have a function that generates a tuple of Four-Momenta (in the right order!) from the first three components - `_construct_moms_from_coords` - and we can use the Julia `...` suffix operator to "unpack" a tuple into a set of function parameters in place.
- providing an implementation of `Base.eltype` (as with FourMomentum) to return the base type T of a particular Event
-  

In [ ]:
# construct event from coordinates
Event(E_in::Real,cos_theta::Real,phi::Real,weight::Real) = #add me

#implement Base.eltype

## Event Generation

Our event generator needs to generate events with some plausible probability distribution. The trivial case would be to apply an entirely uniform distribution; but it would also be useful to provide a set of events distributed by some weighting function.

All of this code belongs to the `event_generation` directory - and we are starting in `serial.jl`. (You might guess where we're going via the filenames.)

### Uniform Distribution

We can use the Julia standard library `rand()` function to generate (pseudo)random numbers - rand has multiple methods with different type signatures, but the one most useful to us is:

 rand(rng::AbstractRNG, nitems::Int)

which uses the supplied (P)RNG to generate an array of nitems values distributed on $[0...1]$.

It's good practice in Julia code that does statistical work to pass the specific (P)RNG to the function, to allow flexibility. The Julia style guide implies that we should put this kind of customisation option first.
We also need to specify the energy of our events, and the number of events we should generate. 

For the energy, we don't have a specific type requirement - but we may need to know what the type chosen *was*, so our function can ensure it is type-stable. As such, we'll declare it with an explicit type **T**, and condition that T as a subtype of Real with a `where` relation after the parameters. 

For the number of events, we know this must be some kind of integer type, so we can explicitly restrict that in the parameter list. 

In order to create an event, we need:
- the input energy, which will be the same for all events. 
- the $\cos\theta$ and $\phi$ for each event
    - uniformly distributed 0n $[-1...1]$ for the first
    - uniformly distributed on $[0...2\pi]$ for the second
- the weight of the event 
    - which we can take from differential_cross_section function from `cross_section.jl`, with the given energy and $\cos\theta$.

We can then call the Event constructor with the energy, angles, weight to generate the event we return.

So, first lets write some code that does this for 1 event.



In [ ]:
function generate_flat_event(rng::AbstractRNG, E_in::T) where {T<:Real}
    cos_theta = 2*rand(rng)-1
    phi = #
    #fill in the rest 

    event = Event(#fill in parameters)
end

The advantage of Julia's built-in broadcasting functionality is that we can simply replace the 'rand(rng)' with 'rand(rng,nevents)' to generate an array of values rather than a single one. Each of our operations in the above function can be replaced with a "dotted" one ('+' $\rightarrow$ '.+' etc), *including the function call parentheses that make the Event(s)*, and it will be seamlessly and efficiently broadcasted over the arrays.

In [ ]:
function generate_flat_events_cpu(rng::AbstractRNG,E_in::T,nevents::Int) where {T<:Real}
    cos_theta = 2.*rand(rng,nevents).-1 
    #fill in the rest

end

### Rejection based sampling

As we mentioned, the other approach to event generation would be to directly use the weights to conditionally reject each event on creation. In this case, the output is an "unweighted" set of events, where their frequency in the array reflects their probability implicitly.

In order to properly normalise our probabilities, we need to determine the highest possible weight we can get - this is when the $\mu^+\mu^-$ pair is emitted colinearly with the $e^+e^-$ pair - that is, when $\cos\theta = -1$. Of course, this is still a function as the distribution is also dependant on the energy scale.

That's pretty easy to write, though:

In [ ]:
max_weight(E_in) =  differential_cross_section(E_in,-1)

Now, our main rejection algorithm can be naïvely written as follows:

- prepare a vector of Event{T}s, where T is the type of our energy parameter of length nevents
(this prevents many small allocations from repeatedly pushing to an empty vector)

- j = 1 
- loop:
    - generate a $\cos\theta$ in uniform distribution (as above)
    - calculate the weight of the candidate event with this energy, $\cos\theta$

    - accept event with probability weight/max_weight (to avoid the division, it's easier to compare weight with rng*max_weight, where rng is a value on $[0..1]$ ).
        - if accepted: 
        - calculate a $\phi$ distributed as above
        - construct an event from the energy, $\cos\theta$, $\phi$, with a weight of one
            - to ensure type stability, the `one` function can be used to generate a unit value of the type of an existing variable
        - assign the event to the jth element of the event array
        - if j = nevents, break
        - increment j
- return the array of events, which is now full of generated items 


In [ ]:
function generate_events_cpu(rng::AbstractRNG,E_in::T,nevents::Int) where {T<:Real}
#fill me in
end

This version of the event generator is fine, but it has a limitation: because we're using the rejection step to condition when we *append* to the array of events, this is an inherently serial algorithm. We *could* parallelise the previous steps, but we would always need to lock the event array and j on appends, which seriously limits our maximum performance.

A more parallelisable approach - although more wasteful of memory - would be to pre-allocate a results array that is much larger than we need, and *always* add each generated event to the array, with a marker indicating if it was accepted or not

The output can then be filtered to return only the accepted events in the array as a final step.

This is clearly problematic, as we cannot know the size of the results array we will need until we've finished calculating events, as the process is fundamentally stochastic.

So, a compromise position is the following:

Pre-allocate a "chunk" array of candidate events, of some large-but-not-excessive size `chunk_size`. 
Generate `chunk_size` events [potentially in parallel], simply marking each one as accepted as rejected.
Filter the chunk of events down to accepted events (and add them to an accepted events array), and count how many we now have.
Repeat the Generate and Filter steps until the total accepted events is equal to the size of the output array.

The Generate step is effectively already half-written - it's just our single-event-generator `generate_flat_event` function, applied over a vector of input energies via broadcasting.

The filter condition is also familiar - it's our acceptance filter from the serial version of the code. 
We can separate this out into a function that takes an Event and returns a boolean result for accept/reject: 

In [ ]:
function event_rejection_filter(event::Event{T}) 
#implement me (you can just copy the test from earlier)
end

Julia actually allows us to apply a bit-vector as a mask on an existing array, resulting in a copy of the elements of the array which are `true` in the bit-vector.

So, for example, given:

```julia
array = [1,2,3,4]
```
then
```julia
mask = iseven.(array) #produces [false,true,false,true]
evens = array[mask] #copies 2 and 4 into evens
```
(you can force Julia to provide references, not copies of elements with the *@views* macro, but we don't need it here - we *do* want to compact our entries into a new array)

Using this, it's clear how our per-chunk iteration should progress.

### The wrapper around the chunk process

Since we'll allow the caller of the generator to specify the Type of the array we want to use for our chunk_array. (This seems unimportant right now, but bear with us until the next section.) Types are first-class in Julia, and thus we can pass them to functions just as any other parameter.

Because there *is* an obvious default ( Vector{T} where T is the type of the energy parameter ) we can provide a default argument here. 
We can also provide a restriction on the ARRAY_TYPE in the `where` guard for the function, as clearly it must be some kind of one-dimensional array type (which Julia calls AbstractVector) with an element of type T. 

Similarly, we should allow the caller to specify the *size* of the chunk_array we're using internally - but we can also specify a reasonable default (say, 100), so this should also not be a mandatory argument. 
(We could limit the type of chunksize to be Integer if we wanted.)

Inside our function we need to:

- create an empty array of Events with element type T - this is our output array
- we can use the sizehint! function on this new array with an Integer size to hint to the JIT that we will be adding items to this array until it reaches the specified size - this allows pre-allocation of memory up-front.

- then, to allow broadcasting effectively, we should make a vector (of the type supplied in the parameters) of Energies for the events we'll generate each chunk.
    - you can use `fill!` to fill an array with identical values.

- then we simply need a loop which:
    - generates via broadcasting, an event for each element of the energy array
    - generates a mask for the event array
    - appends the filtered array to the output array (with `append!` )
    - and updates a count of the elements generated
        - exit if we got to the requested amount



Parallelising the broadcasting

The advantage of Julia's type hierarchy is that this event generator now works for any type of Array, for free.

By default, we've been passing arrays allocated on the heap in system memory, but the CUDA module - and other GPU computing modules - provide Array types which are allocated on GPU memory, and which transparently perform operations on the GPU, in parallel, rather than on the system CPU.

Simply by changing the type of array we allocate, then, we can get GPU acceleration "for free"!

If you have a GPU on your system, then...